# **Enabling GPU in Google Colab**

In [ ]:
import torch
print(torch.cuda.is_available())

True


# **Step 1: Train the MLP Model**


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

# Specifying preprocessing pipeline
categorical_columns = ['Ticker']
numeric_columns = ['Open', 'Close', 'High', 'Low', 'Adjusted Close', 'Volume']

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)
    ])

# Defining model pipeline
model2 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('mlp', MLPRegressor(max_iter=500, random_state=42))
])

# Specifying grid for hyperparameter tuning
param_grid = {
    'mlp__hidden_layer_sizes': [(30,30,30), (50,50,50), (50,100,50)],
    'mlp__activation': ['tanh', 'relu'],
    'mlp__solver': ['sgd', 'adam'],
}

# Using GridSearchCV to find best hyperparameters
grid_search = GridSearchCV(model2, param_grid, cv=5)

# Loading the dataset
data = pd.read_csv('/top_50_stocks_data_formatted.csv')

# Separating features and target
X = data[categorical_columns + numeric_columns]
y = data['Close']

# Splitting data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fitting model
grid_search.fit(X_train, y_train)

# Displaying best parameters
print(f"Best parameters: {grid_search.best_params_}")

# Making predictions
mlp_predictions = grid_search.predict(X_test)

# Calculate MSE
mse_mlp = mean_squared_error(y_test, mlp_predictions)
print(f"MLP Mean Squared Error: {mse_mlp}")

# Plot actual vs predicted
plt.scatter(y_test, mlp_predictions)
plt.plot(y_test, y_test, color='r')
plt.xlabel('Actual Prices')
plt.ylabel('Predicted Prices')
plt.title('MLP: Actual vs Predicted Stock Prices')
plt.show()


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

# **Step 2: Train the RNN Model**


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler

# Preparing data for RNN
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data[['Close']])

# Create a function to process data for RNN input
def create_rnn_data(data, time_steps=60):
    X, y = [], []
    for i in range(time_steps, len(data)):
        X.append(data[i-time_steps:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

time_steps = 60
X_rnn, y_rnn = create_rnn_data(scaled_data, time_steps)

# Reshape data to fit RNN input
X_rnn = np.reshape(X_rnn, (X_rnn.shape[0], X_rnn.shape[1], 1))

# Split data into training and testing sets
split = int(0.8 * len(X_rnn))
X_rnn_train, X_rnn_test = X_rnn[:split], X_rnn[split:]
y_rnn_train, y_rnn_test = y_rnn[:split], y_rnn[split:]

# Build RNN model
model_rnn = Sequential()
model_rnn.add(LSTM(units=50, return_sequences=True, input_shape=(X_rnn_train.shape[1], 1)))
model_rnn.add(LSTM(units=50))
model_rnn.add(Dense(1))

model_rnn.compile(optimizer='adam', loss='mean_squared_error')

# Train the RNN model
model_rnn.fit(X_rnn_train, y_rnn_train, epochs=100, batch_size=32)

# Make predictions
rnn_predictions = model_rnn.predict(X_rnn_test)
rnn_predictions = scaler.inverse_transform(rnn_predictions)

# Calculate MSE
mse_rnn = mean_squared_error(y_rnn_test, rnn_predictions)
print(f"RNN Mean Squared Error: {mse_rnn}")

# Plot actual vs predicted
plt.scatter(y_rnn_test, rnn_predictions)
plt.plot(y_rnn_test, y_rnn_test, color='r')
plt.xlabel('Actual Prices')
plt.ylabel('Predicted Prices')
plt.title('RNN: Actual vs Predicted Stock Prices')
plt.show()


# **Step 3: Combine Predictions (Ensemble)**


In [ ]:
# Combine predictions using simple averaging
ensemble_predictions = (mlp_predictions + rnn_predictions.flatten()) / 2

# Calculate MSE for ensemble
mse_ensemble = mean_squared_error(y_test, ensemble_predictions)
print(f"Ensemble Mean Squared Error: {mse_ensemble}")

# Plot actual vs predicted
plt.scatter(y_test, ensemble_predictions)
plt.plot(y_test, y_test, color='r')
plt.xlabel('Actual Prices')
plt.ylabel('Predicted Prices')
plt.title('Ensemble: Actual vs Predicted Stock Prices')
plt.show()